In [1]:
import requests 

In [2]:
from bs4 import BeautifulSoup 

In [4]:
import re 

In [5]:
def start_requests(url):
    r = requests.get(url)
    return r.content

In [6]:
movie_Dict = {'Movies Name':[],'Release Year':[], 'Genre':[], 'Director':[]}